# MVD 11. cvičení


## 1. část - Collaborative Filtering (item-item)

Vytvořte si matici podle příkladu v přednášce na item-item přístup na snímku 24. Využijte centered cosine similarity pro výpočet podobností stejně jako v přednášce a vyberte dva nejbližší sousedy. Ověřte, že je výsledek v přednášce správný a implementujte funkci tak, aby bylo možné jednoduše spočítat i libovolné další vyhodnocení. 

In [1]:
import numpy as np
from colabFilter import ColabFilter

items =  np.asarray([
    [1,0,3,0,0,5,0,0,5,0,4,0],
    [0,0,5,4,0,0,4,0,0,2,1,3],
    [2,4,0,1,2,0,3,0,4,3,5,0],
    [0,2,4,0,5,0,0,4,0,0,2,0],
    [0,0,4,3,4,2,0,0,0,0,2,5],
    [1,0,3,0,3,0,0,2,0,0,4,0],
])

colabf = ColabFilter(items)
print("dist 0,0:",colabf.dist(0,0))
print("dist 0,1:",colabf.dist(0,1))
print("dist 0,2:",colabf.dist(0,2))
print("dist 0,3:",colabf.dist(0,3))
print("dist 0,4:",colabf.dist(0,4))
print("dist 0,5:",colabf.dist(0,5))
print("closest",colabf.closest())



dist 0,0: 0.9999999999999999
dist 0,1: -0.17854212213729673
dist 0,2: 0.41403933560541256
dist 0,3: -0.10245014273309601
dist 0,4: -0.30895719032666236
dist 0,5: 0.5870395085642741
closest (0.5870395085642741, (0, 5))


In [3]:
out = (colabf.dist(0,2)*2+colabf.dist(0,5)*3)/(colabf.dist(0,2)+colabf.dist(0,5))
out

2.586406866934817

## Bonus - Content-based Filtering

Stáhněte si Kaggle dataset [Spotify Recommendation system](https://www.kaggle.com/bricevergnou/spotify-recommendation). Z datasetu budete potřebovat:

- data.csv = příznaky k jednotlivým skladbám + příznak liked pro klasifikaci

Úkolem je:

1. Načíst data z csv.
2. Vytvořit train (90 %) a test (10 %) split pomocí knihovny [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
3. Vytvořit model logistické regrese (vlastní implementace / využití knihovny sklearn nebo jiné).
4. Vyhodnotit data na testovací sadě (např. metoda score u LogisticRegression).

**Skóre pro uznání úlohy musí být vyšší než 89 %.**

Dobrovolné: vytvořit graf predikovaných vs aktuálních hodnot

Získání bodu navíc: Využijte i další algoritmy pro binární klasifikaci.

In [53]:
import csv
import numpy as np
xl = list()
yl = list()
normFilter = []
with open("data.csv","r") as file:
    reader = csv.reader(file)
    desc = next(reader)
    print(desc)
    l1 = next(reader)
    l1 = [float(el) for el in l1]
    l1[-3]/=60000
    print(l1,np.linalg.norm(l1))
    xl.append(l1[:-1])
    yl.append(l1[-1])
    for line in reader:
        lineF = [float(el) for el in line]
        lineF[-3]/=60000
        xl.append(lineF[:-1])
        yl.append(lineF[-1])
x = np.asarray(xl)
y = np.asarray(yl)
print("x",x.shape)

['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'liked']
[0.803, 0.624, 7.0, -6.764, 0.0, 0.0477, 0.451, 0.000734, 0.1, 0.628, 95.968, 5.0754, 4.0, 0.0] 96.6851211458555
x (195, 13)


In [67]:
import csv
import numpy as np
import tensorflow as ts
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.1)

clf = LogisticRegression(penalty="l2",random_state=40).fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))


0.9314285714285714
0.9


g:\Program Files\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [136]:
import tensorflow as tf
from tensorflow import keras

model = tf.keras.Sequential([tf.keras.Input(shape=(13,)),tf.keras.layers.Normalization(),tf.keras.layers.Dense(13),tf.keras.layers.Dense(1)], name=None)
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss= 'mean_absolute_error')
for i in range(20):
    model.fit(X_train,y_train,epochs =200,verbose=0)
    model.evaluate(X_test,y_test)
    y_pred = model.predict(X_test)
    print(y_pred[:,0])
    i = (y_pred < 0.5)*1
    print(np.sum((i[:,0]==y_test)*1)/len(y_test))

Model: "sequential_67"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_11 (Normaliza  (None, 13)               27        
 tion)                                                           
                                                                 
 dense_185 (Dense)           (None, 13)                182       
                                                                 
 dense_186 (Dense)           (None, 1)                 14        
                                                                 
Total params: 223
Trainable params: 196
Non-trainable params: 27
_________________________________________________________________
1/1 [==============================] - 0s 77ms/step
[ 1.408912    1.0735333   0.5986006   1.3061383  -0.33357024 -0.96812415
  0.8675902   1.4946339   0.74269223  0.7531884  -1.80883    -0.69029117
  0.95205045 -0.09613586  1.3896973   0.6947758   0.33